In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
df = pd.read_csv("data2.csv")
df

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,Ghana,GHA,"Revenue, excluding grants (% of GDP)",GC.REV.XGRT.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,1.452980e+01,1.415894e+01,1.452053e+01,1.506929e+01,1.469481e+01,1.375451e+01,1.492637e+01,1.555055e+01,NaN,NaN
1,Ghana,GHA,"Revenue, excluding grants (current LCU)",GC.REV.XGRT.CN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.666592e+10,3.109227e+10,3.815966e+10,4.650193e+10,5.239349e+10,5.390952e+10,6.891431e+10,9.553267e+10,NaN,NaN
2,Ghana,GHA,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.454000e+01,6.484600e+01,6.530600e+01,6.456500e+01,6.380700e+01,6.236600e+01,6.191200e+01,6.147900e+01,6.147300e+01,61.138
3,Ghana,GHA,"Employment to population ratio, 15+, total (%)...",SL.EMP.TOTL.SP.NE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.453900e+01,NaN,6.893600e+01,NaN,NaN,NaN,5.557400e+01,6.176000e+01,NaN,NaN
4,Ghana,GHA,GDP (current US$),NY.GDP.MKTP.CD,1.217230e+09,1.302674e+09,1.382516e+09,1.540798e+09,1.731296e+09,2.053463e+09,...,4.940601e+10,5.616493e+10,6.040592e+10,6.729891e+10,6.833797e+10,7.004310e+10,7.952442e+10,7.426336e+10,7.637040e+10,NaN
5,Ghana,GHA,GDP (constant 2015 US$),NY.GDP.MKTP.KD,7.657875e+09,7.920515e+09,8.245982e+09,8.609297e+09,8.799505e+09,8.919970e+09,...,4.940601e+10,5.107270e+10,5.522435e+10,5.864830e+10,6.246500e+10,6.278604e+10,6.597335e+10,6.849189e+10,7.050882e+10,NaN
6,Ghana,GHA,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,3.429674e+00,4.109159e+00,4.405974e+00,2.209328e+00,1.368999e+00,...,2.120759e+00,3.373466e+00,8.128895e+00,6.200078e+00,6.507775e+00,5.139417e-01,5.076466e+00,3.817505e+00,2.944786e+00,NaN
7,Ghana,GHA,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,1.169056e+01,1.105833e+01,1.157703e+01,1.224424e+01,1.199700e+01,1.134043e+01,1.224469e+01,1.229754e+01,NaN,NaN
8,Ghana,GHA,Tax revenue (current LCU),GC.TAX.TOTL.CN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.145518e+10,2.428349e+10,3.042420e+10,3.778419e+10,4.277460e+10,4.444777e+10,5.653310e+10,7.554822e+10,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# import pandas as pd
# import numpy as np
from scipy.interpolate import CubicSpline
from datetime import datetime
from scipy.interpolate import interp1d

# # Load your dataset
# df = pd.read_csv('data2.csv')

# Define year columns
year_columns = [col for col in df.columns if '[YR' in col]
years = np.array([int(col.split()[0]) for col in year_columns])

# Indicators to process
selected_indicators = {
    'GC.REV.XGRT.GD.ZS': 'Revenue, excluding grants (% of GDP)',
    'NY.GDP.MKTP.KD': 'GDP (constant 2015 US$)',
    'SL.EMP.TOTL.SP.ZS': 'Employment to population ratio (15+, total %)'
}

# Create monthly time range from Jan 1991 to Dec 2024
monthly_dates = pd.date_range(start='1991-01-01', end='2024-12-01', freq='MS')
monthly_years = monthly_dates.year + (monthly_dates.month - 1) / 12

# Initialize DataFrame
interpolated_data = pd.DataFrame({'date': monthly_dates})

# Interpolate each indicator using cubic spline (with extrapolation)
for code, name in selected_indicators.items():
    row = df[df['Series Code'] == code].iloc[0]
    values = row[year_columns].astype(float).values
    valid_mask = ~np.isnan(values)
    years_valid = years[valid_mask]
    values_valid = values[valid_mask]

    if years_valid.min() <= 1991 and years_valid.max() >= 2022:
        if code == 'GC.REV.XGRT.GD.ZS':
            # Use linear extrapolation for revenue
            interp = interp1d(years_valid, values_valid, kind='linear', fill_value='extrapolate')
            interpolated = interp(monthly_years)
            interpolated = np.maximum(interpolated, 0)  # Clamp to zero to avoid negatives
        else:
            # Use cubic spline for others
            spline = CubicSpline(years_valid, values_valid, extrapolate=True)
            interpolated = spline(monthly_years)

        interpolated_data[name] = interpolated


# Save to CSV
interpolated_data.to_csv('monthly_revenue_employment_GDP_data_1991_2024.csv', index=False)

# Preview
interpolated_data.tail(30)


,date,"Revenue, excluding grants (% of GDP)",GDP (constant 2015 US$),"Employment to population ratio (15+, total %)"
378,2022-07-01,15.862639,6.943176e+10,61.440130
379,2022-08-01,15.914654,6.959465e+10,61.444785
380,2022-09-01,15.966669,6.976257e+10,61.450720
381,2022-10-01,16.018684,6.993673e+10,61.457222
382,2022-11-01,16.070699,7.011835e+10,61.463578
383,2022-12-01,16.122713,7.030864e+10,61.469075
384,2023-01-01,16.174728,7.050882e+10,61.473000
385,2023-02-01,16.226743,7.072011e+10,61.474640
386,2023-03-01,16.278758,7.094372e+10,61.473283
387,2023-04-01,16.330773,7.118087e+10,61.468215


In [15]:
# Load interpolated macroeconomic data (your output file)
macro_df = pd.read_csv('monthly_revenue_employment_GDP_data_1991_2024.csv', parse_dates=['date'])

# Load commodity prices
commodity_df = pd.read_csv('commodity_historical.csv', parse_dates=['date'])

# Merge on date
merged_df = pd.merge(macro_df, commodity_df, on='date', how='left')

# Save merged data
merged_df.to_csv('merged_macro_commodity.csv', index=False)

# Preview
merged_df.head()

,date,"Revenue, excluding grants (% of GDP)",GDP (constant 2015 US$),"Employment to population ratio (15+, total %)",Crude Oil Brent Price,Cocoa Price,Gold Price
0,1991-01-01,14.597519,1.386282e+10,74.834000,23.65,1.2417,383.64
1,1991-02-01,14.372773,1.391355e+10,74.690804,19.40,1.2147,363.83
2,1991-03-01,14.148027,1.396124e+10,74.556498,19.45,1.2103,363.34
3,1991-04-01,13.923281,1.400648e+10,74.430605,19.25,1.1458,358.38
4,1991-05-01,13.698535,1.404985e+10,74.312644,19.30,1.0622,356.95
